In [ ]:
from pathlib import Path
import numpy as np

from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
from aicsimageio.writers import OmeTiffWriter

import sys
src_path = str(Path.cwd().parent.parent)
if src_path not in sys.path:
    sys.path.append(src_path)

import src.d00_utils.utilities as utils

In [ ]:
selected = ['P62-A2']

In [ ]:
prev_train_imgpath = Path(input())

In [ ]:
input_dir = Path(input())

In [ ]:
imgpaths = []

for selection in selected:
    imgpaths.extend([imgpath for imgpath in input_dir.glob(f'*{selection}*')])

print(imgpaths)

In [ ]:
imgs = []

prev_img_file = AICSImage(prev_train_imgpath, reader=OmeTiffReader)
prev_img = prev_img_file.data
imgs.append(prev_img)

for imgpath in imgpaths:
    img_file = AICSImage(imgpath, reader=OmeTiffReader)
    img = img_file.data
    
    if img.shape[0] > 1:
        img_fewtps = np.concatenate([img[0, np.newaxis, :, :, :, :], img[1, np.newaxis, :, :, :, :], img[-1, np.newaxis, :, :, :, :]], axis=0)
        imgs.append(img_fewtps)
    else:
        imgs.append(img)

    

In [ ]:
# Get the smallest y and x dimensions to crop all images to the same size

y_min = None
x_min = None

for img in imgs:
    print(img.shape)
    
    if y_min is None:
        y_min = img.shape[3]
    else:
        y_min = np.minimum(img.shape[3], y_min)
    
    if x_min is None:
        x_min = img.shape[4]
    else:
        x_min = np.minimum(img.shape[4], x_min)

imgs_crop = [img[:, :, :, :y_min, :x_min] for img in imgs]
imgs_crops = imgs

In [ ]:
imgs_stacked = np.concatenate(imgs_crop, axis=0)
print(imgs_stacked.shape)

In [ ]:
ome_metadata = utils.construct_ome_metadata(imgs_stacked, physical_pixel_sizes=prev_img_file.physical_pixel_sizes)

In [ ]:
savename = input("Enter name for stacked image (excluding suffix):")
savedir = Path('/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/ImageJ/training_imgs')
savepath = savedir / (savename + '.ome.tif')

In [ ]:
OmeTiffWriter.save(imgs_stacked, savepath, ome_xml=ome_metadata)